In [89]:
import pandas as pd
import mysql.connector.connection
import traceback
from datetime import datetime
from env.desencriptar import desencriptar
"❌✅"

'❌✅'

CONFIGURACION

In [ ]:
fecha = '2025-04-20'
publicar = 1

2025-04-20 00:00:00


In [91]:
def guardar_en_txt(nombre_archivo, texto):
    with open(nombre_archivo, 'a', encoding='utf-8') as archivo:
        archivo.write(texto + '\n')

def conexion_db(host, user, password, database):
    try:
        cnx = mysql.connector.connect(host=host, user=user, password=password, database=database)
        print("✅ Conexion a base de datos establecida")
        guardar_en_txt("registro_carga.txt", f"[{datetime.strptime(fecha, '%Y-%m-%d')}] - ✅ Se establecio correctamente la conexion a base de datos")
        return cnx
    except Exception as erro_conection:
        print(f"❌ Ocurrio un error al intentar conectarse con la base de datos \
              Error: {erro_conection}")
        guardar_en_txt("registro_carga.txt", f"[{datetime.strptime(fecha, '%Y-%m-%d')}] - ❌ Ocurrio un error al conectarse a base de datos")

def insert_query(cnx, query, data):
    try:
        cursor = cnx.cursor()
        cursor.executemany(query, data)
        cnx.commit()
        print("✅ Se insertaron correctamente los datos en la tabla pivote_tabla_asesores")
        guardar_en_txt("registro_carga.txt", f"[{datetime.strptime(fecha, '%Y-%m-%d')}] - ✅ Los datos fueron insertados correctamente")
    except Exception as error_insercion:
        print(f"❌ Ha ocurrido un error en la insercion de datos {error_insercion}")
        guardar_en_txt("registro_carga.txt", f"[{datetime.strptime(fecha, '%Y-%m-%d')}] - ❌ Ocurrio un error en la insercion de datos")


def data_extract (cnx, query):
        try:
            cursor = cnx.cursor()
            cursor.execute(query)
            resultado = cursor.fetchall()
            if len(resultado) == 0:
                print("⚠️ La consulta de extraccion vino vacia")
                exit()
            else:
                print("✅ Se extrajeron los datos correctamete")
                guardar_en_txt("registro_carga.txt", f"[{datetime.strptime(fecha, '%Y-%m-%d')}] - ✅ Se extrajeron correctamente los datos de db_omnia")

            return resultado
        except Exception as extraction_error:
            print(f"❌ Ocurrio un error al extraer los datos de la base \
                  Error: {extraction_error}")
            guardar_en_txt("registro_carga.txt", f"[{datetime.strptime(fecha, '%Y-%m-%d')}] - ❌ Ocurrio un error a la hora de extraer los datos de db_omnia")

            
def cerrar_conexion(cnx):
    cnx.close()
    print("✅ Se cerro la conexion a base de datos")




In [92]:
query_omnia = f"""
SELECT legajo,
documento, 
apellidos, 
nombres, 
area, 
puesto, 
sitio, 
equipo, 
cliente, 
usuario_neotel
FROM asesores 
WHERE puesto = 'REPRESENTANTE' AND sub_campana = 'PORTABILIDAD' AND fecha_baja is null;"""
columnas = ["legajo",
"documento", 
"apellidos", 
"nombres", 
"area", 
"puesto", 
"sitio", 
"equipo", 
"clientes", 
"usuario_neotel"]

In [93]:
# Tiramos un espacio en el txt para mantener la prolijidad
guardar_en_txt("registro_carga.txt", "\n")
cred = desencriptar()
## Consumimos datos de la tabla db_omnia para realizar la actualizacion a la tabla pivote
cnx = conexion_db(cred['host'], cred['user'], cred['password'], cred['database_omnia'])
result = data_extract(cnx, query_omnia)
cerrar_conexion(cnx)
omnia = pd.DataFrame(result, columns=columnas)
omnia = omnia.astype({"legajo": int, "documento" : int, "usuario_neotel" : int})
omnia["fecha"] = fecha 
omnia.head()

✅ Las credenciales se desencriptaron correctamente
✅ Conexion a base de datos establecida
✅ Se extrajeron los datos correctamete
✅ Se cerro la conexion a base de datos


,legajo,documento,apellidos,nombres,area,puesto,sitio,equipo,clientes,usuario_neotel,fecha
0,1600,19113210,RESTREPO ORDOSGOITIA,ARIANA ESTEFANNY,OPERACIONES,REPRESENTANTE,SAN RAFAEL,"MAGNATERRA, EMILSE JULIETA (40296981)",MOVISTAR,1093,2025-04-20
1,884,19116652,APABLAZA GEREZ,ELIZABETH SABRINA,OPERACIONES,REPRESENTANTE,SAN RAFAEL,"RISELLI, LUCAS (39346102)",MOVISTAR,1173,2025-04-20
2,932,26305314,MORENO,PAOLA BEATRIZ,OPERACIONES,REPRESENTANTE,SAN RAFAEL,"GONZALEZ, ANDREA DENISE (41285468)",MOVISTAR,1157,2025-04-20
3,863,28367210,RUSSO,MARIA VERONICA,OPERACIONES,REPRESENTANTE,SAN RAFAEL,"GONZALEZ, ANDREA DENISE (41285468)",MOVISTAR,1014,2025-04-20
4,684,28634033,CHOQUE,LUCAS RODRIGO,OPERACIONES,REPRESENTANTE,SAN RAFAEL,"MAGNATERRA, EMILSE JULIETA (40296981)",MOVISTAR,1101,2025-04-20


CARGA DB


In [94]:
data = [tuple(row) for row in omnia.values]

# placeholders
placeholder = ', '.join(['%s' for col in omnia.columns])

# columnas_str
columnas_str = ', '.join(omnia.columns)
query_insert_pivote = f"INSERT INTO pivote_tabla_asesores({columnas_str}) VALUES ({placeholder})"

In [95]:
if publicar == 1:    
    cnx = conexion_db(cred['host'], cred['user'], cred['password'], cred['database_porta'])
    insert_query(cnx, query_insert_pivote, data)
    cerrar_conexion(cnx)


✅ Conexion a base de datos establecida
✅ Se insertaron correctamente los datos en la tabla pivote_tabla_asesores
✅ Se cerro la conexion a base de datos
